# 1. Entorno

# 2. Librerías

In [163]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Librería para manipular los servicios de AWS
import boto3

#Utilitario para leer archivos de datos
from sagemaker.inputs import TrainingInput

# 3. Inicio de Sesión sobre SageMaker

In [164]:
#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Verificamos
print(region)

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

us-east-1


# 4. Definición de Ruta de Archivos

In [165]:
#Bucket en donde se encuentran los archivos
#IMPORTANTE: REEMPLAZAR  POR TUS INICIALES
bucket = "datasetsbdajac"

#Lectura de datos de entrenamiento
data = TrainingInput(
    f"s3://{bucket}/data/insurance_dataset/", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

# 5. Configuración del Algoritmo

In [166]:
#Definimos el nombre para nuestro job de entrenamiento
nombreDeJobDeEntrenamiento = "entrenamiento-neuronal"

#Definimos el tipo de predicción que hará el algoritmo
#Para una predicción numérica debemos definir "regressor"
tipoDePrediccion = "regressor"

#Definimos la cantidad de servidores
#Con 1, nuestro algoritmo se ejecutará en 1 servidor
#Con más de 1, nuestro algoritmo se ejecutará en un clúster de varios servidores
numeroDeServidores = 1

#Definimos el tipo de servidor
tipoDeServidor = "ml.m5.large"

# 6. Configuración de las métricas del Algoritmo

In [167]:
#Métrica para elegir el mejor modelo
metricaParaElegirElMejorModelo = "validation:loss"

In [168]:
#Elegimos el modelo con el mayor valor de accuracy
comoSeleccionarElValorDeMetrica = "Minimize"

In [169]:
#Definimos los tipos de métricas que devuelve Tensorflow en sus logs
#De esta manera SageMaker sabe cómo extraer los datos de entrenamiento (con el "Regex" [patrón de extracción])
metricasDisponiblesEnTensorflow = [
    {
        "Name": "validation:loss", 
        "Regex": ".*loss: ([0-9\\.]+) - val_loss: [0-9\\.]+.*"
    }
]

# 7. Definición del entrenador del modelo

In [170]:
#Versión de Python con el que fue construido el script
versionPython = "py38"

In [171]:
#Versión de Tensorflow con el que fue construido el script
versionTensorflow = "2.6"

In [172]:
#Directorio en donde se cuentra el script
directorio = "."

In [173]:
#Nombre del script Tensorflow
scriptTensorflow = "red_neuronal_1.py"

In [174]:
#Estimator para entrenar modelos de Tensorflow
from sagemaker.tensorflow import TensorFlow

In [175]:
#Definimos el entrenador del algoritmo
entrenador = TensorFlow(
    role = rol, #Rol que ejecuta servicios sobre AWS
    py_version = versionPython, #Versión de Python con el que fue construido el script
    framework_version = versionTensorflow, #Versión de Tensorflow con el que fue construido el script
    source_dir = directorio, #Directorio en donde se cuentra el script
    entry_point = scriptTensorflow, #Nombre del script Tensorflow
    metric_definitions = metricasDisponiblesEnTensorflow, #Definimos los tipos de métricas que devuelve Tensorflow en sus logs
    objective_metric_name = metricaParaElegirElMejorModelo, #Métrica para elegir el mejor modelo
    objective_type = comoSeleccionarElValorDeMetrica, #Elegimos el modelo con el mayor valor de accuracy
    instance_count = numeroDeServidores, #Definimos la cantidad de servidores
    instance_type = tipoDeServidor #Definimos el tipo de servidor
)

# 8. Definición de Hyper-Parámetros

In [176]:
#Utilitario para definir rangos de hyper-parámetros que pueden tomar valores continuos
from sagemaker.tuner import ContinuousParameter

In [177]:
#Utilitario para definir rangos de hyper-parámetros que pueden tomar valores enteros
from sagemaker.tuner import IntegerParameter

In [178]:
#Utilitario para definir rangos de hyper-parámetros que pueden tomar valores enteros
from sagemaker.tuner import CategoricalParameter

In [179]:
#Definimos los hyper-parámetros del modelo
hyperparametros = {
    "learning-rate": ContinuousParameter(0.0001, 0.1), #Ratio de aprendizaje
    "batch-size": IntegerParameter(32, 128)  #Tamaño de batch
}

In [180]:
#Definimos la cantidad de procesos de entrenamiento
#Cada proceso de entrenamiento tiene sus propios parámetros
#El output de un proceso de entrenamiento es un modelo entrenado
cantidadDeProcesosDeEntrenamiento = 10 #IMPORTANTE: Generalmente 100

In [181]:
#Definimos la cantidad de procesos de entrenamiento en paralelo para no saturar la infraestructura
cantidadDeProcesosDeEntrenamientoSimultaneos = 10

In [182]:
#Utilitario para crear la malla de hyper-parametrización
from sagemaker.tuner import HyperparameterTuner

In [183]:
#Creamos la malla de hyper-parametrización
mallaDeHyperParametros = HyperparameterTuner(
    entrenador, #Entrenador del algoritmo
    metricaParaElegirElMejorModelo, #Métrica para elegir el mejor modelo
    hyperparametros, #Hyper-parámetros definidos
    metric_definitions = metricasDisponiblesEnTensorflow, #Definimos los tipos de métricas que devuelve Tensorflow en sus logs
    objective_type = comoSeleccionarElValorDeMetrica, #Elegimos el modelo con el mayor valor de accuracy
    max_jobs = cantidadDeProcesosDeEntrenamiento, #Cantidad de procesos de entrenamiento
    max_parallel_jobs = cantidadDeProcesosDeEntrenamientoSimultaneos #Cantidad de procesos de entrenamiento en paralelo
)

In [184]:
#Ejecutamos la malla de hyper-parametrización
#TIEMPO: 5 MINUTOS
mallaDeHyperParametros.fit(inputs = {"training": data})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.............................................!


# 9. Selección del mejor modelo

In [185]:
#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

In [186]:
#Obtenemos el mejor modelo entrenado (el del menor error)
nombreDelMejorModelo = mallaDeHyperParametros.best_training_job()

In [187]:
#Verificamos
print(nombreDelMejorModelo)

tensorflow-training-241006-2352-010-24e61387


In [188]:
#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDelMejorModelo)

#Dentro de la sub-variable "FinalMetricDataList" tenemos las métricas del modelo
descripcionDeEntrenamiento["FinalMetricDataList"]

[{'MetricName': 'validation:loss',
  'Value': 193671040.0,
  'Timestamp': datetime.datetime(2024, 10, 6, 23, 55, 2, tzinfo=tzlocal())},
 {'MetricName': 'ObjectiveMetric',
  'Value': 193671040.0,
  'Timestamp': datetime.datetime(2024, 10, 6, 23, 55, 2, tzinfo=tzlocal())}]

In [189]:
#Extraemos los hyper-parametros del modelo
descripcionDeEntrenamiento["HyperParameters"]

{'_tuning_objective_metric': 'validation:loss',
 'batch-size': '88',
 'learning-rate': '0.06263820444233216',
 'model_dir': '"s3://sagemaker-us-east-1-767828747956/tensorflow-training-2024-10-06-23-52-01-977/model"',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"TensorFlow"',
 'sagemaker_estimator_module': '"sagemaker.tensorflow.estimator"',
 'sagemaker_job_name': '"tensorflow-training-2024-10-06-23-52-01-977"',
 'sagemaker_program': '"red_neuronal_1.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-767828747956/tensorflow-training-2024-10-06-23-52-01-977/source/sourcedir.tar.gz"'}

In [190]:
#Extraemos el learning rate
descripcionDeEntrenamiento["HyperParameters"]["learning-rate"]

'0.06263820444233216'

In [191]:
#Extraemos el batch size
descripcionDeEntrenamiento["HyperParameters"]["batch-size"]

'88'